Hi David, before I forget,  about the PauliOp problem I told you at lunch on Tuesday. This is how I had originally found the error:

`CustomObservables` not parameterized returns an error using pennylane, whereas qiskit works well.

1. Doing:

```

observable = CustomObservable(num_qubits, ["Z", "I"], parameterized=False)
param_obs = []
qnn.evaluate(x_array, [], param_obs, "f")["f"]

```
returns a pennylane error but not a qiskit error.

2. However, doing:

```

observable = CustomObservable(num_qubits, ["Z", "I"], parameterized=True)
param_obs = [(1) for i in range(observable.num_parameters)] # real coefficients
qnn.evaluate(x_array, [], param_obs, "f")["f"]

```

Works fine.

See the two examples below




In [1]:

from squlearn.observables import *
from squlearn.encoding_circuit import QiskitEncodingCircuit, LayeredEncodingCircuit
import numpy as np
from squlearn.qnn.lowlevel_qnn import LowLevelQNN
from squlearn import Executor
from squlearn.observables import CustomObservable
import numpy as np


num_qubits = 1
x_array = np.array([[0.75], 
                    [0.1]])
circuit = LayeredEncodingCircuit(num_qubits, 1, "x", )
circuit.Rx("x") 

observable = CustomObservable(num_qubits, ["Z", "I"], parameterized=True)
param_obs = [(1) for i in range(observable.num_parameters)] # real coefficients
print("Observable \n", observable)

qnn_pennylane = LowLevelQNN(circuit, observable, Executor("pennylane"))
qnn_qiskit = LowLevelQNN(circuit, observable, Executor("qiskit"))


print("Qiskit with real coefficients for the observable ")
print("f ", qnn_qiskit.evaluate(x_array, [], param_obs, "f")["f"])
print("dfdx", qnn_qiskit.evaluate(x_array, [], param_obs, "dfdx")["dfdx"])
print("dfdop", qnn_qiskit.evaluate(x_array, [], param_obs, "dfdop")["dfdop"])

print("Pennylane with real coefficients for the observable") 
print("f", qnn_pennylane.evaluate(x_array, [], param_obs, "f")["f"])
print("dfdx", qnn_pennylane.evaluate(x_array, [], param_obs, "dfdx")["dfdx"])
print("dfdop", qnn_pennylane.evaluate(x_array, [], param_obs, "dfdop")["dfdop"])


Observable 
 SparsePauliOp(['Z', 'I'],
              coeffs=[ParameterExpression(1.0*p[0]), ParameterExpression(1.0*p[1])])
Qiskit with real coefficients for the observable 
f  [1.73168887 1.99500417]
dfdx [[-0.68163876]
 [-0.09983342]]
dfdop [[0.73168887 1.        ]
 [0.99500417 1.        ]]
Pennylane with real coefficients for the observable
f [1.73168887 1.99500417]
dfdx [[-0.68163876]
 [-0.09983342]]
dfdop [[0 1]
 [0 1]]


In [34]:
from squlearn.observables import *
from squlearn.encoding_circuit import QiskitEncodingCircuit, ChebyshevRx
import numpy as np
from squlearn.qnn.lowlevel_qnn import LowLevelQNN
from squlearn import Executor
from squlearn.observables import CustomObservable
import numpy as np


num_qubits = 1
x_array = np.array([[0.75], 
                    [0.1]])
circuit = ChebyshevRx(num_features=1, num_qubits=num_qubits, num_layers=1)

observable = SummedPaulis(num_qubits)

qnn_pennylane = LowLevelQNN(circuit, observable, Executor("pennylane", shots=500000))
qnn_qiskit = LowLevelQNN(circuit, observable, Executor("pennylane"))

qnn_pennylane = LowLevelQNN(circuit, observable, Executor("qiskit", shots=50000))
qnn_qiskit = LowLevelQNN(circuit, observable, Executor("qiskit"))


param = np.random.rand(qnn_pennylane.num_parameters)
param_obs = np.random.rand(observable.num_parameters)


print("Qiskit with real coefficients for the observable ")
#print("f ", qnn_qiskit.evaluate(x_array, param, param_obs, "f")["f"])
#print("dfdx", qnn_qiskit.evaluate(x_array, param, param_obs, "dfdx")["dfdx"])
print("dfdxdp", qnn_qiskit.evaluate(x_array, param, param_obs, "dfdxdp")["dfdxdp"])
print("dfdxdx", qnn_qiskit.evaluate(x_array, param, param_obs, "dfdxdx")["dfdxdx"])


print("-----------------")
print("Pennylane shots") 
#print("f", qnn_pennylane.evaluate(x_array, param, param_obs, "f")["f"])
#print("dfdx", qnn_pennylane.evaluate(x_array, param, param_obs, "dfdx")["dfdx"])
print("dfdxdp", qnn_pennylane.evaluate(x_array, param, param_obs, "dfdxdp")["dfdxdp"])
print("dfdxdx", qnn_qiskit.evaluate(x_array, param, param_obs, "dfdxdx")["dfdxdx"])



Qiskit with real coefficients for the observable 
dfdxdp [[[ 1.0322133 ]
  [ 0.38786098]]

 [[ 0.49214824]
  [-0.05793016]]]
dfdxdx [[[0.61090914]]

 [[0.10133526]]]
-----------------
Pennylane shots
dfdxdp [[[ 1.03227187]
  [ 0.38809707]]

 [[ 0.49092537]
  [-0.05776457]]]
dfdxdx [[[0.61090914]]

 [[0.10133526]]]


In [5]:
from squlearn.observables import *
from squlearn.encoding_circuit import QiskitEncodingCircuit, ChebyshevRx
import numpy as np
from squlearn.qnn.lowlevel_qnn import LowLevelQNN
from squlearn import Executor
from squlearn.observables import CustomObservable
import numpy as np


num_qubits = 2
x_array = np.array([[0.75], 
                    [0.1]])

circuit = ChebyshevRx(num_features=1, num_qubits=num_qubits, num_layers=1)
observable = SummedPaulis(num_qubits)

qnn_pennylane_shots = LowLevelQNN(circuit, observable, Executor("pennylane", shots=500000))
qnn_pennylane_statevector = LowLevelQNN(circuit, observable, Executor("pennylane"))

qnn_qiskit_shots = LowLevelQNN(circuit, observable, Executor("qiskit", shots=50000))
qnn_qiskit = LowLevelQNN(circuit, observable, Executor("qiskit"))
#qnn_pennylane = LowLevelQNN(circuit, observable, Executor("qiskit", shots=50000))
#qnn_qiskit = LowLevelQNN(circuit, observable, Executor("qiskit"))

np.random.seed(1)
param = np.random.rand(qnn_pennylane_shots.num_parameters)
param_obs = np.random.rand(observable.num_parameters)


print("Pennylane statevector ")
print("f\n", qnn_pennylane_statevector.evaluate(x_array, param, param_obs, "f")["f"])
print("dfdx\n", qnn_pennylane_statevector.evaluate(x_array, param, param_obs, "dfdx")["dfdx"])
print("dfdxdp\n", qnn_pennylane_statevector.evaluate(x_array, param, param_obs, "dfdxdp")["dfdxdp"])
#print("dfdxdx\n", qnn_pennylane_statevector.evaluate(x_array, param, param_obs, "dfdxdx")["dfdxdx"])
#print("dfdxdop\n", qnn_pennylane_statevector.evaluate(x_array, param, param_obs, "dfdxdop")["dfdxdop"])


print("-----------------")
print("Pennylane shots") 
print("f \n", qnn_pennylane_shots.evaluate(x_array, param, param_obs, "f")["f"])
print("dfdx \n", qnn_pennylane_shots.evaluate(x_array, param, param_obs, "dfdx")["dfdx"])
print("dfdp \n", qnn_pennylane_shots.evaluate(x_array, param, param_obs, "dfdxdp")["dfdxdp"])
#print("dfdxdx \n", qnn_pennylane_shots.evaluate(x_array, param, param_obs, "dfdxdx")["dfdxdx"])
#print("dfdxdop\n", qnn_pennylane_shots.evaluate(x_array, param, param_obs, "dfdxdop")["dfdxdop"])


print("-----------------")
print("qiskit") 
print("f \n", qnn_qiskit.evaluate(x_array, param, param_obs, "f")["f"])
print("dfdx \n", qnn_qiskit.evaluate(x_array, param, param_obs, "dfdx")["dfdx"])
print("dfdxdp \n", qnn_qiskit.evaluate(x_array, param, param_obs, "dfdpdx")["dfdpdx"])
#print("dfdxdx \n", qnn_qiskit.evaluate(x_array, param, param_obs, "dfdxdx")["dfdxdx"])


print("-----------------")
print("qiskit shots") 
print("f \n", qnn_qiskit_shots.evaluate(x_array, param, param_obs, "f")["f"])
print("dfdx \n", qnn_qiskit_shots.evaluate(x_array, param, param_obs, "dfdx")["dfdx"])
print("dfdxdp \n", qnn_qiskit_shots.evaluate(x_array, param, param_obs, "dfdpdx")["dfdpdx"])
#print("dfdxdx \n", qnn_qiskit_shots.evaluate(x_array, param, param_obs, "dfdxdx")["dfdxdx"])



Pennylane statevector 
f
 [0.3558851  0.25388108]
dfdx
 [[0.18300929]
 [0.1394669 ]]
dfdxdp
 [[[ 0.16170338  0.27386335  0.08768908  0.10615746]]

 [[ 0.03015719  0.11875324 -0.03103286 -0.02106788]]]
-----------------
Pennylane shots
f 
 [0.35573124 0.25390571]
dfdx 
 [[0.18294305]
 [0.13935763]]
dfdp 
 [[[ 0.16118708  0.27380725  0.08779801  0.10648818]]

 [[ 0.03018952  0.11853509 -0.03115682 -0.0210164 ]]]
-----------------
qiskit
f 
 [0.3558851  0.25388108]
dfdx 
 [[0.18300929]
 [0.1394669 ]]
dfdxdp 
 [[[ 0.16170338]
  [ 0.27386335]
  [ 0.08768908]
  [ 0.10615746]]

 [[ 0.03015719]
  [ 0.11875324]
  [-0.03103286]
  [-0.02106788]]]
-----------------
qiskit shots
f 
 [0.3566699  0.25505732]
dfdx 
 [[0.18409977]
 [0.1393096 ]]
dfdxdp 
 [[[ 0.16198549]
  [ 0.27556979]
  [ 0.08785197]
  [ 0.10598586]]

 [[ 0.03097671]
  [ 0.11857496]
  [-0.03107528]
  [-0.02144533]]]
